In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing  import LabelEncoder

import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.layers import Embedding
from tensorflow.python.keras.layers import Conv1D
from tensorflow.python.keras.layers import MaxPooling1D
from tensorflow.python.keras.layers import Lambda
from tensorflow.python.keras.layers import GlobalAveragePooling1D
from keras.layers import Input, Embedding, LSTM, Dense, Bidirectional
from keras.models import Model
from keras import backend as K
from keras.models import Sequential

tfds.disable_progress_bar()

In [2]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [3]:
# load Dataset1.
url_data = pd.read_csv('urldata.csv')
url_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450176 entries, 0 to 450175
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  450176 non-null  int64 
 1   url         450176 non-null  object
 2   label       450176 non-null  object
 3   result      450176 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 13.7+ MB


In [4]:
url_data.head()

,Unnamed: 0,url,label,result
0,0,https://www.google.com,benign,0
1,1,https://www.youtube.com,benign,0
2,2,https://www.facebook.com,benign,0
3,3,https://www.baidu.com,benign,0
4,4,https://www.wikipedia.org,benign,0


In [5]:
number = LabelEncoder()
url_data['label'] = number.fit_transform(url_data['label'].astype(str))
url_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450176 entries, 0 to 450175
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  450176 non-null  int64 
 1   url         450176 non-null  object
 2   label       450176 non-null  int64 
 3   result      450176 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 13.7+ MB


In [6]:
train, test = train_test_split(url_data, test_size=0.8)
train, val = train_test_split(train, test_size=0.2)

train.to_csv('train.csv', header=False)
test.to_csv('test.csv', header=False)

print("Shape of x_train: ", train.shape)
print("Shape of x_valid: ", test.shape)

Shape of x_train:  (72028, 4)
Shape of x_valid:  (360141, 4)


In [13]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
types  = [tf.string, tf.int64]
record_defaults = ['XXXX', 999] # Only provide defaults for the selected columns
tf.executing_eagerly()


True

In [14]:
train_dataset = tf.data.experimental.CsvDataset("train.csv", record_defaults, select_cols=[2, 3])
train_dataset.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int32, name=None))

In [15]:
test_dataset = tf.data.experimental.CsvDataset("test.csv", record_defaults, select_cols=[2, 3])
test_dataset.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int32, name=None))

In [18]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [19]:
def normlize(text):
    result = tf.strings.lower(text)
    result = tf.strings.split(text, sep='/').numpy()
    return result
# @tf.function
# def sequence(string,maxlen=200):
#     tokens = []
#     for c in string:
#         if not c in printable:
#             continue
#         tokens.append(printable.index(c))
#     tokens = tokens[-maxlen:]
#     if len(tokens) < maxlen:
#         tokens = [0]*(maxlen-len(tokens))+tokens
#     return tokens

In [20]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b'https://www.bradfordmdschoolsfa.co.uk/Chapter3.htm'
 b'https://www.nicekicks.com/2011/05/air-jordan-12-gary-payton-pe-2/'
 b'https://www.facebook.com/pages/Marathon-Oasis-de-Montreal/123974107673472']

labels:  [0 0 0]


In [21]:
train_dataset.element_spec

(TensorSpec(shape=(None,), dtype=tf.string, name=None),
 TensorSpec(shape=(None,), dtype=tf.int32, name=None))

In [22]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda url, label: url))

# encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
#   standardize=normlize,
#   max_tokens=VOCAB_SIZE,
#   output_sequence_length=VOCAB_SIZE)
# encoder.adapt(train_dataset.map(lambda url, label: url))


In [23]:
vocab = np.array(encoder.get_vocabulary())
len(vocab)

1000

In [25]:
for sample_url, label in train_dataset.take(1):
  print('text: ', sample_url.numpy())
  print('label: ', label.numpy())

text:  [b'https://www.zoolex.org/'
 b'http://e-assistance-freemobile.net/moncompte_identification/mobile-info-recouvrement_facture/e36aa77cb8d88365a3d6bf0ddd6b7fc0/c7e447e7d82df77573ec5b2e8bfec3ba/myaccount/home.php?customerid=59703'
 b'https://www.sportsillustrated.cnn.com/vault/topic/article/Thomas_Blake/1900-01-01/2100-12-31/cdd/1/index.htm'
 b'http://dovecotstudios.com/security-confirme-your-information/info/'
 b'http://geekgamergirl.com/wp-admin/America/AOL/index-aol.html'
 b'https://www.amazon.com/Long-Walk-Stephen-King/dp/0451196716'
 b'https://www.worldfilm.about.com/od/independentfilm/Independent_Film.htm'
 b'https://www.urbantoronto.ca/forum/showthread.php/3338-New-TTC-Subway-Cars/page142'
 b'https://www.ehow.com/list_6045624_tours-montreal_-canada.html'
 b'http://blog.teatroacorte.it/jakes/index.htm'
 b'https://www.partnersfcu.org/partners-federal-credit-union/current-news/'
 b'https://www.myspace.com/danaandmarla' b'https://www.make-a-wave.org/'
 b'http://teknosi.com/878hf3

In [26]:
def getconvmodel(filter_length,nb_filter):
    model = Sequential()
    model.add(Conv1D(filters=nb_filter,
                        input_shape=(1000, 200),
                        kernel_size=filter_length,
                        padding='same',
                        activation='relu'))
    model.add(Lambda(sum_1d, output_shape=(nb_filter,)))
    #model.add(BatchNormalization(mode=0))
    model.add(Dropout(0.5))
    return model

In [27]:
def sum_1d(X):
    return K.sum(X, axis=1)


In [28]:
# RNN Model with LSTM.
input_shape=(len(vocab),)
main_input = Input(shape=(1,), dtype='string', name='main_input')

rnn_model = tf.keras.Sequential([
    main_input,
    encoder,
    tf.keras.layers.Embedding(
        output_dim=200,
        input_dim=1000,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, input_shape=(None, len(vocab)))),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])




NotImplementedError: Cannot convert a symbolic Tensor (bidirectional/forward_lstm/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [ ]:
# CNN Model.

input_shape=(len(vocab),)

main_input = Input(shape=(1,), dtype='string', name='main_input')
x = encoder(main_input)
embedding_layer =  tf.keras.layers.Embedding(
                    output_dim=200,
                    input_dim=1000,
                    mask_zero=True);
x = embedding_layer(x)
conv1 = getconvmodel(2,256) (x)
conv2 = getconvmodel(3,256) (x)
conv3 = getconvmodel(4,256) (x)
conv4 = getconvmodel(5,256) (x)

concatted = tf.keras.layers.Concatenate()([conv1, conv2, conv3, conv4])

middle = Dense(1024,activation='relu')(concatted)
middle = Dropout(0.5)(middle)

middle = Dense(1024,activation='relu')(middle)
middle = Dropout(0.5)(middle)

output = Dense(1,activation='sigmoid')(middle)

cnn_model = Model(inputs=main_input,outputs=output)
cnn_model.summary()

In [ ]:
model = rnn_model

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
model.summary()

In [ ]:
# Working model.

# main_input = Input(shape=(200,), dtype='int32', name='main_input')
# input_shape=(1000,1)

# model = tf.keras.Sequential([
#     encoder,
#     tf.keras.layers.Embedding(
#                     output_dim=200,
#                     input_dim=1000,
#                     mask_zero=True),
#     tf.keras.layers.Conv1D(filters=256,
#                           kernel_size=4,
#                           input_shape=(1000,),
#                           activation='relu',
#                           bias_initializer='random_uniform',
#                           padding='same'),
#     tf.keras.layers.Lambda(sum_1d, output_shape=(nb_filter,)),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dense(1)
# ])

# Working model -2

# input_shape=(len(vocab),)

# main_input = Input(shape=(1,), dtype='string', name='main_input')
# x = encoder(main_input)
# embedding_layer =  tf.keras.layers.Embedding(
#                     output_dim=200,
#                     input_dim=1000,
#                     mask_zero=True);
# x = embedding_layer(x)
# conv1 = getconvmodel(2,256) (x)
# conv2 = getconvmodel(3,256) (x)
# conv3 = getconvmodel(4,256) (x)
# conv4 = getconvmodel(5,256) (x)

# concatted = tf.keras.layers.Concatenate()([conv1, conv2, conv3, conv4])

# middle = Dense(1024,activation='relu')(concatted)
# middle = Dropout(0.5)(middle)

# middle = Dense(1024,activation='relu')(middle)
# middle = Dropout(0.5)(middle)

# output = Dense(1,activation='sigmoid')(middle)

# model = Model(inputs=main_input,outputs=output)
# model.summary()

In [ ]:
#   tf.keras.layers.Conv1D(filters=256,
#                           kernel_size=4,
#                           activation='relu',
#                           bias_initializer='random_uniform',
#                           padding='same'),
#     tf.keras.layers.Lambda(sum_1d),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Conv1D(filters=256,
#                           kernel_size=5,
#                           activation='relu',
#                           bias_initializer='random_uniform',
#                           padding='same'),
#     tf.keras.layers.Lambda(sum_1d),
#     tf.keras.layers.Dropout(0.5,
#                           input_shape=input_shape),

In [ ]:
print([layer.name for layer in model.layers])

In [ ]:
sample_url = ('https://www.facebook.com/')
predictions = model.predict(np.array([sample_url]))
print(predictions[0])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
# history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

In [ ]:
# plt.figure(figsize=(8, 4))
# plt.subplot(1, 2, 1)
# plot_graphs(history, 'accuracy')
# plt.ylim(0.7, 1)
# plt.subplot(1, 2, 2)
# plot_graphs(history, 'loss')
# plt.ylim(0, 0.6)